In [718]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from joblib import dump, load
import joblib

## 0. DataFrame

In [719]:
football_df = pd.read_csv('data/all_data_with_elo.csv', low_memory = False)
football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,WHD,WHA,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO
0,0,D1,2003/8/1,Bayern Munich,Ein Frankfurt,3.0,1.0,H,17.0,6.0,...,4.5,10.00,NaN,NaN,NaN,NaN,NaN,NaN,1859.379272,1593.249268
1,1,F1,2003/8/1,Lille,Lyon,1.0,0.0,H,NaN,NaN,...,3.0,2.20,NaN,NaN,NaN,NaN,NaN,NaN,1612.968018,1726.539795
2,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,3.1,5.00,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045
3,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,3.1,2.40,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732
4,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,3.5,5.00,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38428,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,3.2,2.20,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584
38429,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,3.0,2.80,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872
38430,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,3.2,2.75,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956
38431,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,3.0,3.40,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542


## 1. Descriptive Statistics 

**1.1 DataFrame Shape**

In [720]:
# no. rows and no. cols
football_df.shape

(38433, 29)

In [721]:
# feature names
print(football_df.columns.tolist())

['Unnamed: 0', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'AHh', 'B365AHH', 'B365AHA', 'AHCh', 'B365CAHH', 'B365CAHA', 'HomeTeamELO', 'AwayTeamELO']


**1.2 NaN Values**

In [722]:
football_df.isnull().sum()

Unnamed: 0         0
Div                0
Date               0
HomeTeam           0
AwayTeam           0
FTHG               0
FTAG               0
FTR                0
HS              1762
AS              1762
HST             2568
AST             2568
B365H             49
B365D             49
B365A             49
IWH             1976
IWD             1976
IWA             1976
WHH              573
WHD              573
WHA              573
AHh              262
B365AHH          276
B365AHA          276
AHCh           28479
B365CAHH       28481
B365CAHA       28481
HomeTeamELO      125
AwayTeamELO      126
dtype: int64

In [723]:
# total elements in 
football_df.size

1114557

In [724]:
# total number of NaN
football_df.size - football_df.count().sum()

102960

In [725]:
# total number of NaN rows
football_df.isnull().any(axis = 1).sum()

30386

In [726]:
# total number of NaN columns
football_df.isnull().any(axis = 0).sum()

21

## 2. Data Wrangling and Feature Transformation/Development

### 2.1 NaN Handling

`TODO`: drop NaN values along columns: {Date, Home Team, Away Team, FTR} <br>
`TODO`: identify betting odds w/ most available data

In [727]:
# 当前方法仅提取这几个字段 分区 日期 主队 客队 full-time-result 三家机构的胜平负 主队ELO评分 客队ELO评分
# nan_mask = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 
#             'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'AHh', 'B365AHH', 'B365AHA', 'HomeTeamELO', 'AwayTeamELO']
nan_mask = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'B365H', 'B365D', 'B365A', 'WHH', 'WHD', 'WHA', 'AHh', 'B365AHH', 'B365AHA', 'HomeTeamELO', 'AwayTeamELO']

In [728]:
nan_football_df = football_df.dropna(subset = nan_mask)
nan_football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,WHD,WHA,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO
2,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,3.1,5.00,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045
3,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,3.1,2.40,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732
4,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,3.5,5.00,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298
5,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,3.4,4.00,-0.75,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972
7,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,3.3,4.50,-0.75,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38428,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,3.2,2.20,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584
38429,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,3.0,2.80,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872
38430,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,3.2,2.75,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956
38431,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,3.0,3.40,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542


In [729]:
conditions = [
    nan_football_df['AHh'] >= 2.5,         # AHh >= 2.75
    (nan_football_df['AHh'] >= 1.5) & (nan_football_df['AHh'] <= 2.25),  # 1.75 < AHh <= 2.75
    (nan_football_df['AHh'] >= 0.25) & (nan_football_df['AHh'] <= 1.25),  # 0.25 < AHh <= 1.75
    nan_football_df['AHh'] == 0,            # AHh == 0
    (nan_football_df['AHh'] >= -1.25) & (nan_football_df['AHh'] <= -0.25),  # -1.75 < AHh <= -0.25
    (nan_football_df['AHh'] >= -2.25) & (nan_football_df['AHh'] <= -1.5),  # -2.75 < AHh <= -1.75
    nan_football_df['AHh'] <= -2.5
]
# easy_conditions = [
#     nan_football_df['AHh'] <= -0.25,
#     nan_football_df['AHh'] == 0,
#     nan_football_df['AHh'] >= 0.25,
# ]
labels = [3, 2, 1, 0, -1, -2, -3]
# easy_labels = [-1, 0, 1]

nan_football_df['balance_val'] = np.select(conditions, labels)
nan_football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,WHA,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val
2,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,5.00,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1
3,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,2.40,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0
4,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,5.00,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1
5,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,4.00,-0.75,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1
7,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,4.50,-0.75,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38428,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,2.20,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1
38429,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,2.80,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0
38430,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,2.75,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0
38431,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,3.40,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1


In [730]:
nan_football_df['asia_final_result'] = nan_football_df['FTHG'] - nan_football_df['FTAG'] + nan_football_df['balance_val']
nan_football_df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val,asia_final_result
2,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1,-2.0
3,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0,-1.0
4,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1,-4.0
5,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,-0.75,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1,-4.0
7,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,-0.75,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38428,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1,0.0
38429,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0,1.0
38430,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0,0.0
38431,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1,1.0


In [731]:
nan_football_df_noNone = nan_football_df.dropna(subset = nan_mask)
nan_football_df_noNone

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val,asia_final_result
2,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1,-2.0
3,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0,-1.0
4,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1,-4.0
5,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,-0.75,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1,-4.0
7,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,-0.75,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38428,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1,0.0
38429,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0,1.0
38430,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0,0.0
38431,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1,1.0


In [732]:
nan_football_df_noNone.reset_index(inplace=True, drop=True)
nan_football_df_noNone

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val,asia_final_result
0,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,-0.75,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1,-2.0
1,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,0.00,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0,-1.0
2,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,-0.75,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1,-4.0
3,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,-0.75,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1,-4.0
4,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,-0.75,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,0.25,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1,0.0
37342,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,0.00,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0,1.0
37343,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,0.00,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0,0.0
37344,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,-0.25,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1,1.0


In [733]:
conditions = [
    nan_football_df_noNone['asia_final_result'] < 0,
    nan_football_df_noNone['asia_final_result'] == 0,
    nan_football_df_noNone['asia_final_result'] > 0,
]
easy_labels = [0, 1, 2]

nan_football_df_noNone['easy_label'] = np.select(conditions, easy_labels)
nan_football_df_noNone

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val,asia_final_result,easy_label
0,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1,-2.0,0
1,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0,-1.0,0
2,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1,-4.0,0
3,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1,-4.0,0
4,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1,0.0,1
37342,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0,1.0,2
37343,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0,0.0,1
37344,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1,1.0,2


In [734]:
# resize shape
football_df.shape[0] - nan_football_df_noNone.shape[0]

1087

### 2.2 Feature Encoding <br>
* $\phi(Date)$ $\Rightarrow$ one column for *year*, second column for *month*, third column for *day of year*
* One hot encode Division, Home and Away Teams
* Label encode Full Time Result (Win/Draw/Loss)

In [735]:
feats = nan_mask
feats.append('easy_label')
feats.append('balance_val')

In [736]:
nan_football_df_noNone

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,...,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,HomeTeamELO,AwayTeamELO,balance_val,asia_final_result,easy_label
0,2,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,NaN,NaN,...,2.050,1.850,NaN,NaN,NaN,1702.604858,1611.196045,-1,-2.0,0
1,3,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,NaN,NaN,...,1.925,1.975,NaN,NaN,NaN,1685.016113,1665.625732,0,-1.0,0
2,4,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,10.0,16.0,...,1.800,2.100,NaN,NaN,NaN,1718.566284,1649.805298,-1,-4.0,0
3,5,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,23.0,19.0,...,2.025,1.875,NaN,NaN,NaN,1719.916748,1692.120972,-1,-4.0,0
4,7,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,NaN,NaN,...,1.900,2.000,NaN,NaN,NaN,1697.354004,1539.958130,-1,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,38428,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,10.0,16.0,...,1.990,1.940,0.25,1.73,2.08,1667.065796,1772.048584,1,0.0,1
37342,38429,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,17.0,12.0,...,1.910,1.990,0.00,1.82,2.11,1635.277222,1704.093872,0,1.0,2
37343,38430,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,3.0,5.0,...,1.890,2.010,-0.25,1.98,1.95,1621.804321,1637.579956,0,0.0,1
37344,38431,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,18.0,5.0,...,1.950,1.980,-0.25,1.98,1.95,1682.297119,1628.680542,-1,1.0,2


In [737]:
learning_df_feat = nan_football_df_noNone.copy()[feats]
learning_df_feat

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.00,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.40,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.00,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.00,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.50,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,3.300,3.300,2.250,3.30,3.2,2.20,0.25,1.990,1.940,1667.065796,1772.048584,1,1
37342,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,2.750,2.900,2.880,2.75,3.0,2.80,0.00,1.910,1.990,1635.277222,1704.093872,2,0
37343,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,2.630,3.000,3.000,2.62,3.2,2.75,0.00,1.890,2.010,1621.804321,1637.579956,1,0
37344,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,2.250,3.000,3.600,2.25,3.0,3.40,-0.25,1.950,1.980,1682.297119,1628.680542,2,-1


In [738]:
learning_df_feat.reset_index(inplace=True, drop=True)
# 保存文件作为历史文件
learning_df_feat.to_csv('.\prediction_data/history_data_balance.csv', index=False, encoding='utf-8-sig')
learning_df_feat

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,5.00,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,2.40,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,5.00,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,4.00,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,4.50,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,I1,2025/1/26,Udinese,Roma,1.0,2.0,A,3.300,3.300,2.250,3.30,3.2,2.20,0.25,1.990,1.940,1667.065796,1772.048584,1,1
37342,SP1,2025/1/26,Vallecano,Girona,2.0,1.0,H,2.750,2.900,2.880,2.75,3.0,2.80,0.00,1.910,1.990,1635.277222,1704.093872,2,0
37343,SP1,2025/1/27,Alaves,Celta,1.0,1.0,D,2.630,3.000,3.000,2.62,3.2,2.75,0.00,1.890,2.010,1621.804321,1637.579956,1,0
37344,I1,2025/1/27,Genoa,Monza,2.0,0.0,H,2.250,3.000,3.600,2.25,3.0,3.40,-0.25,1.950,1.980,1682.297119,1628.680542,2,-1


**2.2.1 Division and Home/Away Team Encoding**

In [739]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

div_encoder = OneHotEncoder()
home_encoder = OneHotEncoder()
away_encoder = OneHotEncoder()

In [740]:
onehot_div = div_encoder.fit_transform(learning_df_feat.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.fit_transform(learning_df_feat.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.fit_transform(learning_df_feat.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

In [741]:
# 保存编码器到本地
joblib.dump(div_encoder, 'div_encoder.pkl')
joblib.dump(home_encoder, 'home_encoder.pkl')
joblib.dump(away_encoder, 'away_encoder.pkl')

['away_encoder.pkl']

In [742]:
learning_df_div = pd.concat([learning_df_feat, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
learning_df_div.drop(columns = ['Div'], inplace = True)

In [743]:
learning_df_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003/8/2,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,...,0,0,0,0,0,0,0,0,0,0
1,2003/8/2,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,...,0,0,0,0,0,0,0,0,0,0
2,2003/8/2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,...,0,0,0,0,0,0,0,0,0,0
3,2003/8/2,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,...,0,0,1,0,0,0,0,0,0,0
4,2003/8/2,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,2025/1/26,Udinese,Roma,1.0,2.0,A,3.300,3.300,2.250,3.30,...,0,0,0,0,0,0,0,0,0,0
37342,2025/1/26,Vallecano,Girona,2.0,1.0,H,2.750,2.900,2.880,2.75,...,0,0,0,0,0,0,0,0,0,0
37343,2025/1/27,Alaves,Celta,1.0,1.0,D,2.630,3.000,3.000,2.62,...,0,0,0,0,0,0,0,0,0,0
37344,2025/1/27,Genoa,Monza,2.0,0.0,H,2.250,3.000,3.600,2.25,...,0,0,0,0,0,0,0,0,0,0


**2.2.2 Full Time Result Encoding**

In [744]:
target_encoder = LabelEncoder()
learning_df_div['Result'] = target_encoder.fit_transform(learning_df_div.easy_label) 
learning_df_div['Result_FTR'] = target_encoder.fit_transform(learning_df_div.FTR)

**2.2.3 Date Encoding**

In [745]:
learning_df_div['Year'] = pd.DatetimeIndex(learning_df_div.Date).year

learning_df_div['Month'] = pd.DatetimeIndex(learning_df_div.Date).month
learning_df_div['Sin_Month'] = np.sin(2*np.pi*learning_df_div.Month/12)
learning_df_div['Cos_Month'] = np.cos(2*np.pi*learning_df_div.Month/12)

learning_df_div['DayofYear'] = pd.DatetimeIndex(learning_df_div.Date).dayofyear
learning_df_div['Sin_Day'] = np.sin(2*np.pi*learning_df_div.DayofYear/365)
learning_df_div['Cos_Day'] = np.cos(2*np.pi*learning_df_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
learning_df = learning_df_div.drop(columns = ['Date','Month'])
# learning_df.drop(columns = ['Date'], inplace = True)

In [746]:
learning_df

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,WHH,WHD,...,AwayTeam 205,AwayTeam 206,Result,Result_FTR,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,1.66,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,2.60,3.1,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,1.57,3.5,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,1.72,3.4,...,0,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,1.66,3.3,...,0,0,0,1,2003,-0.866025,-0.500000,214,-0.516062,-0.856551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,1.0,2.0,A,3.300,3.300,2.250,3.30,3.2,...,0,0,1,0,2025,0.500000,0.866025,26,0.432776,0.901502
37342,Vallecano,Girona,2.0,1.0,H,2.750,2.900,2.880,2.75,3.0,...,0,0,2,2,2025,0.500000,0.866025,26,0.432776,0.901502
37343,Alaves,Celta,1.0,1.0,D,2.630,3.000,3.000,2.62,3.2,...,0,0,1,1,2025,0.500000,0.866025,27,0.448229,0.893919
37344,Genoa,Monza,2.0,0.0,H,2.250,3.000,3.600,2.25,3.0,...,0,0,2,2,2025,0.500000,0.866025,27,0.448229,0.893919


In [747]:
# For Test


### 2.3 Feature Engineering <br>
* $\phi(x)$ feature transformation $\Rightarrow$ last match result, win/loss streak to date, wins to season date
* $\phi(x)$ feature engineering $\Rightarrow$ average the home, away, and draw odds

**2.3.1 Last Match Result** <br>
Indicate the result from the last match played between both teams

# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.easy_label.shift(1, fill_value='Na')
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    return df

In [748]:
def compute_last_n_matches(df, n=5):
    unique_matchups = list(set(zip(df.HomeTeam, df.AwayTeam)))
    df['Last 5 Match Results'] = np.nan  # 新增一列用于存储过去 5 场比赛的结果
    
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        
        # 获取过去 n 场比赛的结果
        # last_n_results = [matchup_df.FTR.shift(i, fill_value='Na') for i in range(1, n+1)]
        last_n_results = [matchup_df.easy_label.shift(i, fill_value='Na') for i in range(1, n+1)]
        
        # 将计算得到的过去 n 场比赛的结果合并为一个字符串或列表，取决于需求
        # 这里使用字符串形式：'result1/result2/...'
        matchup_df['Last 5 Match Results'] = pd.DataFrame(last_n_results).T.apply(lambda x: '/'.join(x), axis=1)
        
        # 将计算得到的结果更新回原始 df 中
        df.loc[matchup_df.index, 'Last 5 Match Results'] = matchup_df['Last 5 Match Results']
    
    # 对 Last 5 Match Results 列进行标签编码
    lmr_encoder = LabelEncoder()
    df['Last 5 Match Results'] = lmr_encoder.fit_transform(df['Last 5 Match Results'])
    
    # 删除原始的 FTR 列
    df.drop(columns=['easy_label'], inplace=True)
    
    return df


In [749]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        # 因为easy_label 不适合作为上次比较结果
        last_match_result = matchup_df.Result.shift(1, fill_value=3)
        # last_match_result = matchup_df.Result_FTR.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    df.drop(columns = ['FTR'], inplace = True)
    return df
learning_df = compute_last_matches(learning_df)
# learning_df.drop(columns = ['FTR'], inplace = True)

In [750]:
learning_df

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,AwayTeam 206,Result,Result_FTR,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.00,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.40,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.00,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.00,...,0,0,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.50,...,0,0,1,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,1.0,2.0,3.300,3.300,2.250,3.30,3.2,2.20,...,0,1,0,2025,0.500000,0.866025,26,0.432776,0.901502,1
37342,Vallecano,Girona,2.0,1.0,2.750,2.900,2.880,2.75,3.0,2.80,...,0,2,2,2025,0.500000,0.866025,26,0.432776,0.901502,1
37343,Alaves,Celta,1.0,1.0,2.630,3.000,3.000,2.62,3.2,2.75,...,0,1,1,2025,0.500000,0.866025,27,0.448229,0.893919,2
37344,Genoa,Monza,2.0,0.0,2.250,3.000,3.600,2.25,3.0,3.40,...,0,2,2,2025,0.500000,0.866025,27,0.448229,0.893919,0


**2.3.2 Home and Away Win/Loss Streak** <br>
Important note about this feature: the win/loss streak is the teams *home* and *away* win streak, *not* its ***consecutive*** win/loss streak.

In [751]:
# https://stackoverflow.com/questions/52976336/compute-winning-streak-with-pandas
# https://joshdevlin.com/blog/calculate-streaks-in-pandas/

In [752]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 1)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 1)
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            home_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(home_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['HomeWin'] == 1:  # 如果上一场比赛主队赢
                    home_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    home_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['HomeWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
            # team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            # team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            # year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            away_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(away_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['AwayWin'] == 1:  # 如果上一场比赛主队赢
                    away_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    away_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['AwayWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
            # team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            # team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            # year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])

In [753]:
learning_df = compute_winstreak(learning_df)
learning_df

,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,...,Result_FTR,Year,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.00,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
1,Guingamp,Marseille,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.40,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
2,Hamburg,Hannover,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.00,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.00,...,0,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
4,Lens,Le Mans,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.50,...,1,2003,-0.866025,-0.500000,214,-0.516062,-0.856551,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,Udinese,Roma,1.0,2.0,3.300,3.300,2.250,3.30,3.2,2.20,...,0,2025,0.500000,0.866025,26,0.432776,0.901502,1,1,0
37342,Vallecano,Girona,2.0,1.0,2.750,2.900,2.880,2.75,3.0,2.80,...,2,2025,0.500000,0.866025,26,0.432776,0.901502,1,1,1
37343,Alaves,Celta,1.0,1.0,2.630,3.000,3.000,2.62,3.2,2.75,...,1,2025,0.500000,0.866025,27,0.448229,0.893919,2,0,0
37344,Genoa,Monza,2.0,0.0,2.250,3.000,3.600,2.25,3.0,3.40,...,2,2025,0.500000,0.866025,27,0.448229,0.893919,0,0,0


**2.3.4 Season Home/Away Wins to Date** <br>
Indicate the number of wins for a team as home and away to date within current season

toy = learning_df[(learning_df.Year == 2010) & (learning_df.HomeTeam == 'Barcelona')][['HomeTeam', 'AwayTeam', 'Result']]
toy['HomeWin'] = toy.Result.replace([0, 1, 2], [0, 0, 1])
toy['HomeWinsToDate'] = toy.HomeWin.cumsum()

In [754]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        if year > 2024:
            # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
            year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
            year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
            
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['HomeWinsToDate'] = team_df.HomeWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['HomeWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
    
            # team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            # year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['AwayWinsToDate'] = team_df.AwayWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['AwayWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
            # team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            # year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])

In [755]:
learning_df = compute_winstodate(learning_df)
learning_df.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)

In [756]:
# learning_df
learning_df

,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,WHA,AHh,B365AHH,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,1.727,3.100,4.500,1.66,3.1,5.00,-0.75,2.050,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,0.0,1.0,2.500,2.875,2.625,2.60,3.1,2.40,0.00,1.925,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,0.0,3.0,1.571,3.500,5.000,1.57,3.5,5.00,-0.75,1.800,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,0.0,3.0,1.833,3.200,3.750,1.72,3.4,4.00,-0.75,2.025,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,0.0,0.0,1.571,3.250,5.500,1.66,3.3,4.50,-0.75,1.900,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,1.0,2.0,3.300,3.300,2.250,3.30,3.2,2.20,0.25,1.990,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,2.0,1.0,2.750,2.900,2.880,2.75,3.0,2.80,0.00,1.910,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,1.0,1.0,2.630,3.000,3.000,2.62,3.2,2.75,0.00,1.890,...,2025,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,2.0,0.0,2.250,3.000,3.600,2.25,3.0,3.40,-0.25,1.950,...,2025,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [757]:
# 保存为pkl文件
learning_df.to_pickle('E:/Data/PKL/learning_df.pkl')

**2.3.5 Website Odds** <br>
The `betting odds` recorded by various betting websites offer insight into sentiment surrounding the outcome of a particular game. 

In [758]:
# betting_feats = ['B365H', 'B365D', 'B365A', 'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', "AHh", "B365AHH", "B365AHA"]
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [759]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]

In [760]:
learning_df = compute_meanodds(learning_df, betting_feats)

### 2.4 Peek @ Learning DataFrame

In [761]:
learning_df

,HomeOdds,AwayOdds,DrawOdds,FTHG,FTAG,B365H,B365D,B365A,WHH,WHD,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,4.500,3.100,1.0,2.0,1.727,3.100,4.500,1.66,3.1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,2.500,2.625,2.875,0.0,1.0,2.500,2.875,2.625,2.60,3.1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,1.571,5.000,3.500,0.0,3.0,1.571,3.500,5.000,1.57,3.5,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,1.833,3.750,3.200,0.0,3.0,1.833,3.200,3.750,1.72,3.4,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,1.571,5.500,3.250,0.0,0.0,1.571,3.250,5.500,1.66,3.3,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,3.300,2.250,3.300,1.0,2.0,3.300,3.300,2.250,3.30,3.2,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,2.750,2.880,2.900,2.0,1.0,2.750,2.900,2.880,2.75,3.0,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,2.630,3.000,3.000,1.0,1.0,2.630,3.000,3.000,2.62,3.2,...,2025,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,2.250,3.600,3.000,2.0,0.0,2.250,3.000,3.600,2.25,3.0,...,2025,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [762]:
learning_df.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG', 'Result_FTR'], inplace = True)

# 3. Model Development

* Establish a baseline Logistic Regression model fit over the entire learning dataframe without special regard to *division* and *team*. 
* Train model over 16 seasons, and predict for the remaining 3 seasons (approximate 80-20 split)

### 3.1 Train and Test Split

In [763]:
split = 0.80
no_seasons = 20

print('No. seasons to train over: ' + str(round(split*no_seasons)))

No. seasons to train over: 16


In [764]:
X, y = learning_df.loc[:, learning_df.columns != 'Result'], learning_df[['Result']]

In [765]:
# full_feat = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result',
#              'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats

# exclude_feats = ['HomeWinsToDate', 'AwayWinsToDate', 'Last Match Result'] 

In [766]:
# X = X[X.columns[~X.columns.isin(exclude_feats)]]
# X

In [767]:
X

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,-1,1,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,-1,0,...,2003,-0.866025,-0.500000,-0.516062,-0.856551,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,3.300,3.300,2.250,0.25,1.990,1.940,1667.065796,1772.048584,1,0,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,2.750,2.900,2.880,0.00,1.910,1.990,1635.277222,1704.093872,0,0,...,2025,0.500000,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,2.630,3.000,3.000,0.00,1.890,2.010,1621.804321,1637.579956,0,0,...,2025,0.500000,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,2.250,3.000,3.600,-0.25,1.950,1.980,1682.297119,1628.680542,-1,0,...,2025,0.500000,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


In [768]:
y

,Result
0,0
1,0
2,0
3,0
4,0
...,...
37341,1
37342,2
37343,1
37344,2


In [769]:
split_year = 2024
start_year = split_year - 15

In [770]:
# 切分训练集和测试集
xTr, xTe = X[(X.Year <= split_year) & (X.Year >= start_year)], X[X.Year > split_year]
yTr, yTe = y.loc[xTr.index, :], y.loc[xTe.index, :]

### 3.2 Normalization <br>
Following our various feature transformations and development, we arrived to a sparse dataframe with the exception of a few features(*Year, DayofYear*). It will be important to *normalize* these features as they are in gross magnitudes compared to the remaining features. During model training, having dominating features (in scale relative to others) can be dangerous as the weight updates may mistakengly favor these larger-scale features because it will have the largest influence on the target output. 

In [771]:
# minmax_scaler.fit_transform()：这个方法首先拟合数据，即计算数据的最小值和最大值，这些值用于后续的缩放。然后，它将这些参数用于转换数据，将原始数据缩放到0和1之间。
# minmax_scaler.transform()：这个方法使用在训练数据上计算得到的最小值和最大值来转换测试数据。这确保了训练数据和测试数据使用相同的缩放标准。
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
xTr.loc[:, ['Year']] = minmax_scaler.fit_transform(xTr.loc[:, ['Year']])
xTe.loc[:, ['Year']] = minmax_scaler.transform(xTe.loc[:, ['Year']])
# 保存到文件
import time
local_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
joblib.dump(minmax_scaler, f'minmax_scaler_{local_time}.pkl')  # 保存为 .pkl 文件

['minmax_scaler_2025_01_31_00_44_24.pkl']

In [772]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats

xTr.loc[:, to_scale] = std_scaler.fit_transform(xTr.loc[:, to_scale])
xTe.loc[:, to_scale] = std_scaler.transform(xTe.loc[:, to_scale])
# 保存到文件
import time
local_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
joblib.dump(std_scaler, f'std_scaler_{local_time}.pkl')  # 保存为 .pkl 文件

['std_scaler_2025_01_31_00_44_24.pkl']

In [773]:
xTr

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
8436,-0.803793,1.973446,2.704354,-2.00,2.03,1.79,2.228498,0.358517,-2,0,...,0.0,5.000000e-01,0.866025,5.161967e-02,0.998667,0,0,0,0.0,0.0
8437,-0.320754,-0.443158,-0.318145,0.00,1.56,2.35,0.784912,1.553909,0,0,...,0.0,5.000000e-01,0.866025,5.161967e-02,0.998667,2,0,0,0.0,0.0
8438,-0.178993,-0.521113,-0.422369,0.00,1.69,2.04,0.114531,0.344419,0,0,...,0.0,5.000000e-01,0.866025,6.880243e-02,0.997630,0,0,0,0.0,0.0
8439,-0.425763,-0.443158,-0.161809,-0.50,1.90,1.96,0.414210,-0.112525,-1,0,...,0.0,5.000000e-01,0.866025,6.880243e-02,0.997630,1,0,0,0.0,0.0
8440,-0.320754,-0.521113,-0.292089,0.00,1.55,2.37,0.539067,0.616487,0,0,...,0.0,5.000000e-01,0.866025,6.880243e-02,0.997630,2,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37160,-0.425763,-0.131338,-0.266033,-0.50,1.95,1.95,0.973010,0.526419,-1,0,...,1.0,-2.449294e-16,1.000000,6.432491e-16,1.000000,2,1,0,7.0,8.0
37161,-0.635780,0.102527,0.619872,-1.00,1.87,2.03,0.799789,-0.655835,-1,0,...,1.0,-2.449294e-16,1.000000,6.432491e-16,1.000000,2,2,1,8.0,7.0
37162,-0.530771,-0.209293,0.098751,-0.75,1.95,1.95,-0.952264,-0.627471,-1,0,...,1.0,-2.449294e-16,1.000000,6.432491e-16,1.000000,3,1,0,1.0,8.0
37163,1.989432,0.609234,-0.834055,1.25,2.02,1.88,-0.865015,1.659679,1,0,...,1.0,-2.449294e-16,1.000000,6.432491e-16,1.000000,3,0,0,3.0,6.0


In [774]:
xTe

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Year,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
37165,1.989432,0.609234,-0.828843,1.25,1.92,2.01,0.396470,2.522054,1,0,...,1.066667,0.5,0.866025,0.017213,0.999852,1,0,0,0.0,0.0
37166,-0.268250,-0.521113,-0.318145,-0.25,1.91,2.02,-0.142761,-0.227654,-1,0,...,1.066667,0.5,0.866025,0.051620,0.998667,2,0,0,0.0,0.0
37167,2.251954,0.414347,-0.834055,1.25,1.93,2.00,-0.640181,2.220230,1,0,...,1.066667,0.5,0.866025,0.051620,0.998667,2,0,0,0.0,0.0
37168,-0.725037,1.193896,1.010712,-1.50,1.92,1.98,0.946354,-0.514325,-2,0,...,1.066667,0.5,0.866025,0.068802,0.997630,0,0,0,0.0,0.0
37169,-0.515020,-0.170315,0.033611,-0.75,1.99,1.91,0.727408,0.182551,-1,0,...,1.066667,0.5,0.866025,0.068802,0.997630,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37341,0.309297,-0.521113,-0.617790,0.25,1.99,1.94,-0.177963,0.712415,1,0,...,1.066667,0.5,0.866025,0.432776,0.901502,1,1,0,1.0,0.0
37342,0.020523,-0.832932,-0.453637,0.00,1.91,1.99,-0.448016,0.135851,0,0,...,1.066667,0.5,0.866025,0.432776,0.901502,1,1,1,1.0,1.0
37343,-0.042482,-0.754977,-0.422369,0.00,1.89,2.01,-0.562472,-0.428489,0,0,...,1.066667,0.5,0.866025,0.448229,0.893919,2,0,0,0.0,0.0
37344,-0.241998,-0.754977,-0.266033,-0.25,1.95,1.98,-0.048568,-0.503997,-1,0,...,1.066667,0.5,0.866025,0.448229,0.893919,0,0,0,0.0,0.0


### 3.3 HomeWins Baseline Model

In [775]:
from sklearn.metrics import accuracy_score

In [776]:
xTr.shape

(28729, 438)

In [777]:
xTe.shape

(181, 438)

In [778]:
# training score
baseline_Tr = np.full((xTr.shape[0], 1), 2) 
accuracy_score(yTr.Result.values, baseline_Tr.ravel())

0.34675763166138746

In [779]:
# testing score
baseline_preds_Te = np.full((xTe.shape[0]  , 1), 2) #predicts home wins all the time
accuracy_score(yTe.Result.values, baseline_preds_Te.ravel())

0.34806629834254144

### 3.4 Multinomial Logistic Regression

**3.4.1** $l2$ Regularized

In [780]:
from sklearn.linear_model import LogisticRegression
l2_lr = LogisticRegression(max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [781]:
# training score
accuracy_score(yTr.Result.values, l2_lr.predict(xTr))

0.4883915207629921

In [782]:
# testing score
lr_preds = l2_lr.predict(xTe)
accuracy_score(yTe.Result.values, lr_preds)

0.5414364640883977

**3.4.1** $l2$ Penalty Tuning

In [783]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

logistic_params = {'C':[0.001,0.01,0.10]}

# logistic_randsearch = RandomizedSearchCV(estimator=LogisticRegression(max_iter=10000),
#                                          param_distributions=logistic_params,
logistic_randsearch = GridSearchCV(estimator=LogisticRegression(max_iter=10000),
                                         param_grid=logistic_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         cv=5,
                                         n_jobs=-1)

logistic_rand_results = logistic_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (logistic_rand_results.best_score_, logistic_rand_results.best_params_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best: 0.471475 using {'C': 0.01}


In [784]:
l2_rs = logistic_rand_results.best_estimator_

In [785]:
# training score
accuracy_score(yTr.Result.values, l2_rs.predict(xTr))

0.4800027846426955

In [786]:
# testing score
accuracy_score(yTe.Result.values, l2_rs.predict(xTe))

0.5248618784530387

**3.4.4** $l1$ Regularized

In [787]:
l1_lr = LogisticRegression(penalty='l1', solver='saga', max_iter = 10000, n_jobs=-1).fit(xTr, yTr.values.ravel())

In [788]:
# training score
accuracy_score(yTr.Result.values, l1_lr.predict(xTr))

0.4868947753141425

In [789]:
# testing score
l1_preds = l1_lr.predict(xTe)
accuracy_score(yTe.Result.values, l1_preds)

0.5303867403314917

**3.4.5** Penalty Tuning

In [790]:
l1_params = {'C':[0.001,0.01,0.10]}

# l1_randsearch = RandomizedSearchCV(estimator=LogisticRegression(penalty='l1',solver='saga', max_iter=10000),
#                                          param_distributions=l1_params,
l1_randsearch = GridSearchCV(estimator=LogisticRegression(penalty='l1',solver='saga', max_iter=10000),
                                         param_grid=l1_params,
                                         scoring='accuracy',
                                         verbose=1,
                                         n_jobs=-1,
                                         cv=5)

l1_rand_results = l1_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (l1_rand_results.best_score_, l1_rand_results.best_params_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best: 0.471997 using {'C': 0.1}


In [791]:
l1_rs = l1_randsearch.best_estimator_ #LogisticRegression(C=0.10, solver='saga', max_iter=10000).fit(xTr, yTr.values.ravel())#

In [792]:
# training score
accuracy_score(yTr.Result.values, l1_rs.predict(xTr))

0.47815795885690415

In [793]:
# testing score
accuracy_score(yTe.Result.values, l1_rs.predict(xTe))

0.5082872928176796

### 3.5 Support Vector Machine

In [794]:
from sklearn.svm import SVC
svm = SVC(max_iter=100000).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, svm.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, svm.predict(xTe))

In [ ]:
predict_svm = svm.predict(xTe)
series_svm = pd.Series(predict_svm, name='Predicted')
compare_result = pd.concat([series_svm, yTe.reset_index()], axis=1)
compare_result

In [ ]:
import matplotlib.pyplot as plt
result_subset = compare_result.tail(200)
plt.figure(figsize=(10,10))
result_subset.plot(x='index', y=['Predicted', 'Result'], kind='line')
plt.title("Prediction vs Real")
plt.show()

In [ ]:
import time
local_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
dump(svm, f'./sklearn_svm_{local_time}.joblib')

**3.5.2** Penalty Tuning

In [ ]:
svm_params = {'C':[0.001,0.01,0.10]}

# svm_randsearch = RandomizedSearchCV(estimator=SVC(max_iter=100000),
#                                          param_distributions=svm_params,
svm_randsearch = GridSearchCV(estimator=SVC(max_iter=100000),
                                         param_grid=svm_params,
                                         scoring='accuracy',
                                         verbose=2,
                                         cv=5,
                                         n_jobs=-1)

svm_rand_results = svm_randsearch.fit(xTr, yTr.values.ravel())
print("Best: %f using %s" % (svm_rand_results.best_score_, svm_rand_results.best_params_))

In [ ]:
svm_rs = svm_rand_results.best_estimator_

In [ ]:
# training score
accuracy_score(yTr.Result.values, svm_rs.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, svm_rs.predict(xTe))

In [ ]:
import time
local_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
dump(svm_rs, f'./sklearn_svm_randsearch_{local_time}.joblib')

### 3.6 Simple Neural Network ####

In [ ]:
# from sklearn.neural_network import MLPClassifier
# mlp = MLPClassifier(hidden_layer_sizes=(512,128,32),
#                     activation='relu',
#                     batch_size=512,
#                     max_iter=10000,
#                     learning_rate_init=1e-4,
#                     early_stopping=True,
#                     alpha=1e-3,
#                    ).fit(xTr, yTr.values.ravel())
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(256,128,32),
                    activation='relu',
                    batch_size=256,
                    max_iter=10000,
                    learning_rate_init=1e-4,
                    early_stopping=False,
                    alpha=1e-2,
                   ).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, mlp.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, mlp.predict(xTe))

In [ ]:
xTr

In [ ]:
yTr

In [ ]:
xTe

In [ ]:
yTe

In [ ]:
predict_val = mlp.predict(xTe)
series_pre = pd.Series(predict_val, name='Predicted')
compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
compare_result

In [ ]:
import matplotlib.pyplot as plt
result_subset = compare_result.tail(200)
plt.figure(figsize=(10,10))
result_subset.plot(x='index', y=['Predicted', 'Result'], kind='line')
plt.title("Prediction vs Real")
plt.show()

In [ ]:
# xTr = xTr.drop(columns=['pad1', 'pad2', 'pad3'])
# xTe = xTe.drop(columns=['pad1', 'pad2', 'pad3'])

### 保存模型

In [ ]:
import time
local_time = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
dump(mlp, f'./sklearn_mlp_{local_time}.joblib')

In [ ]:
# ### 加载模型
# # model_name = 'sklearn_mlp_' + local_time + '.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_21_22_41_15.joblib'
# mlp = load(model_name)

### 3.7 Stacked Classifier ###

In [ ]:
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import StackingClassifier
stacked_clf = StackingClassifier(estimators=[('svm', SVC(max_iter=100000)), ('logistic', LogisticRegression(C=0.01, max_iter=10000))],
                                final_estimator=LogisticRegression(max_iter=10000),
                                n_jobs=-1).fit(xTr, yTr.values.ravel())

In [ ]:
# training score
accuracy_score(yTr.Result.values, stacked_clf.predict(xTr))

In [ ]:
# testing score
accuracy_score(yTe.Result.values, stacked_clf.predict(xTe))

## 4. Result Analysis ##

In [ ]:
## TODO: breakdown results across divisions and/or teams; i.e., see how model performs individually at subgroups

## 5. Scrap Code ##

In [ ]:
barcelona_df = learning_df[(learning_df['HomeTeam 17'] == 1) | (learning_df['AwayTeam 17'] == 1)]
barcelona_df

In [ ]:
bxTr = xTr[(xTr['HomeTeam 17'] == 1) | (xTr['AwayTeam 17'] == 1)]
bxTe = xTe[(xTe['HomeTeam 17'] == 1) | (xTe['AwayTeam 17'] == 1)]

In [ ]:
byTr, byTe = yTr.loc[bxTr.index,:], yTe.loc[bxTe.index,:]

In [ ]:
# training score
accuracy_score(byTr, l1_lr.predict(bxTr))

In [ ]:
# testing score
accuracy_score(byTe, l1_lr.predict(bxTe))

In [ ]:
# training score
accuracy_score(byTr, l2_lr.predict(bxTr))

In [ ]:
# testing score
accuracy_score(byTe, l2_lr.predict(bxTe))

## 6. Pytorch MLP ##

type(xTr)

xTr.shape

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Parameter(torch.randn(feature_dim))

    def forward(self, x):
        # 应用注意力权重
        weights = F.softmax(self.attention_weights, dim=0)
        # 加权求和
        x = x * weights
        return x
    
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features=xTr.shape[1], out_features=512)
        self.bn1 = nn.BatchNorm1d(num_features=512)
        self.attention = Attention(512)
        self.dropout1 = nn.Dropout(p=0.2)
        
        self.fc2 = nn.Linear(in_features=512, out_features=128)
        self.bn2 = nn.BatchNorm1d(num_features=128)
        self.dropout2 = nn.Dropout(p=0.2)
        
        self.fc3 = nn.Linear(in_features=128, out_features=32)
        self.bn3 = nn.BatchNorm1d(num_features=32)
        self.dropout3 = nn.Dropout(p=0.2)
        
        self.fc4 = nn.Linear(in_features=32, out_features=3)  # 输出层改为3，对应三个类别

    def forward(self, x):
        x = self.dropout1(torch.relu(self.bn1(self.fc1(x))))
        x = self.attention(x)
        x = self.dropout2(torch.relu(self.bn2(self.fc2(x))))
        x = self.dropout3(torch.relu(self.bn3(self.fc3(x))))
        x = self.fc4(x)
        return x

# 数据预处理
scaler = StandardScaler()
xTr_scaled = scaler.fit_transform(xTr)
xTr_tensor = torch.tensor(xTr_scaled, dtype=torch.float32).to(device)
yTr_tensor = torch.tensor(yTr.values.ravel(), dtype=torch.long).to(device)

# 创建数据加载器
dataset = TensorDataset(xTr_tensor, yTr_tensor)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

# 创建模型实例
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

train_start = time.time()
# 训练模型
model.train()
for epoch in range(500):  # 假设训练200个epoch
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # 清除之前的梯度

        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        running_loss += loss.item() * inputs.size(0)  # 累计损失
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100  # 计算准确率

    # 每个epoch结束后输出
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
print(f'训练时长： {time.time() - train_start}s')

# 假设 xTe 和 yTe 是 pandas DataFrame 或 Series
# 数据预处理
xTe_scaled = scaler.fit_transform(xTe)  # 使用与训练数据相同的标准化参数
xTe_tensor = torch.tensor(xTe_scaled, dtype=torch.float32).to(device)
yTe_tensor = torch.tensor(yTe.values.ravel(), dtype=torch.long).to(device)

# 创建数据加载器
test_dataset = TensorDataset(xTe_tensor, yTe_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 设置模型为评估模式
model.eval()

# 初始化用于计算准确率的变量
correct = 0
total = 0

# 不计算梯度，因为在评估模式下不需要进行反向传播
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

## 7. Pytorch Transformer ##

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.model_dim = input_dim  # 通常情况下，模型维度与输入维度相同

        # Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.model_dim,
            nhead=num_heads,
            dim_feedforward=512,  # 前馈网络的维度
            dropout=dropout,
            batch_first=True
        )

        # Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 输出层
        self.output_layer = nn.Linear(self.model_dim, self.num_classes)

        # Batch Normalization
        self.bn = nn.BatchNorm1d(self.model_dim)

    def forward(self, x):
        # 增加一个假的序列维度
        x = x.unsqueeze(1)
        # Transformer Encoder
        x = self.transformer_encoder(x)

        # Batch Normalization
        x = self.bn(x[:, 0, :])  # 取序列的第一个元素进行批量归一化

        # 输出层
        x = self.output_layer(x)
        return x

# 补充维度
n_samples_xTr = xTr.shape[0]
n_samples_xTe = xTe.shape[0]
for i in range(1, 4):  # 从 1 到 3，因为需要添加三列
    xTr[f'pad{i}'] = 0  # 添加填充列，初始化为 0
    xTe[f'pad{i}'] = 0  # 添加填充列，初始化为 0

# 参数设置
input_dim = xTr.shape[1]  # 输入特征的维度
num_classes = 3  # 类别数
num_heads = 10  # 注意力头的数量
num_layers = 3  # Transformer层的数量
dropout = 0.8  # Dropout比率

# 创建模型
model = TransformerModel(input_dim, num_classes, num_heads, num_layers, dropout).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 数据加载
# 假设 xTr 和 xTe 已经是适当的 torch.Tensor 对象
xTr_values = xTr.values.astype(float)
xTe_values = xTe.values.astype(float)

xTr_tensor = torch.tensor(xTr_values, dtype=torch.float32).to(device)
xTe_tensor = torch.tensor(xTe_values, dtype=torch.float32).to(device)
yTr_tensor = torch.tensor(yTr.values, dtype=torch.long).to(device).squeeze(1)
yTe_tensor = torch.tensor(yTe.values, dtype=torch.long).to(device).squeeze(1)
# 转换为 one-hot 编码
yTr_tensor = F.one_hot(yTr_tensor, num_classes=num_classes).float()
yTe_tensor = F.one_hot(yTe_tensor, num_classes=num_classes).float()

# 数据加载器
train_dataset = TensorDataset(xTr_tensor, yTr_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=False)

test_dataset = TensorDataset(xTe_tensor, yTe_tensor)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

train_start = time.time()
# 训练模型
model.train()
for epoch in range(500):  # 运行更多的 epoch 以获得更好的结果
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)  # 累计损失
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        _, truth = torch.max(labels.data, 1)
        total += truth.size(0)
        correct += (predicted == truth).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100  # 计算准确率

    # 每个epoch结束后输出
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
print(f'训练时长： {time.time() - train_start}s')

# 设置模型为评估模式
model.eval()

# 初始化用于计算准确率的变量
correct = 0
total = 0

# 不计算梯度，因为在评估模式下不需要进行反向传播
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, truth = torch.max(labels.data, 1)
        total += labels.size(0)
        correct += (predicted == truth).sum().item()

# 计算准确率
accuracy = correct / total
print(f'Accuracy on test set: {accuracy * 100:.2f}%')